# Topic clustering using LDA

### 1. Clean data and return tokens.

In [ ]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


###  Find the meanings of words, synonyms,antonyms using WordNet

In [ ]:
import nltk
nltk.download('wordnet')



In [ ]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)


In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

### Prepare text for topic clustering

In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens


For starters, we have a transcription that contains a discussion about product design.

In [ ]:
import random
text_data = []
with open('UIUX.txt') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        print(tokens)
        text_data.append(tokens)

* First, we are creating a dictionary from the data, then converting to a bag-of-words corpus and saving the dictionary and corpus for future use.
* We are using Gensim for this particular task.

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)


In [ ]:
corpus = [dictionary.doc2bow(text) for text in text_data]

### At present, we are limiting number of topics per mixture to 5

In [ ]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30)
ldamodel.save('model5.gensim')

In [ ]:


topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)



* We have been able to generate top 5 topic mixtures from our collection of transcripts.
* We can safely say that all these topics refer to aspects of product design in some way or the other.


### Running the model after combining all transcripts

In [ ]:
with open('bigtranscript.txt') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        text_data.append(tokens)

In [ ]:
dictionary = corpora.Dictionary(text_data)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [ ]:
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30)
ldamodel.save('model0.gensim')

In [ ]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

#### This is how the summary of all topics present inside the transcripts looks like. Enough to give us a general idea of what these interviews tend to discuss about.

### Further Work

* Try lda2vec and compare results with lda.
* Assign labels to topics generated by the unsupervised learning model.